In [2]:
import torch
from torchvision import models, transforms
from PIL import Image
#import subprocess
#import textwrap
#import json


In [3]:
# ---------- Configuration ----------
OLLAMA_MODEL = "qwen:1.8b"   # the local model installed
IMAGE_PATH = "sample_chest_xray.jpg"

# ---------- Image Preprocessing ----------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [4]:
def load_image(path):
    img = Image.open(path).convert("RGB")
    return transform(img).unsqueeze(0)

In [5]:
image_tensor = load_image(IMAGE_PATH)

In [6]:
image_tensor

tensor([[[[-2.1179, -2.1179, -2.1179,  ..., -2.0837, -2.0837, -2.1008],
          [-2.1179, -2.1179, -2.1179,  ..., -2.0837, -2.0837, -2.1008],
          [-2.1179, -2.1179, -2.1179,  ..., -2.0837, -2.0837, -2.1008],
          ...,
          [-1.2617, -1.3473, -1.3987,  ...,  0.1768,  0.1939,  0.1768],
          [-1.2617, -1.3473, -1.4158,  ...,  0.3138,  0.3138,  0.1597],
          [-1.2274, -1.3815, -1.4329,  ...,  0.1768,  0.0741,  0.1254]],

         [[-2.0357, -2.0357, -2.0357,  ..., -2.0007, -2.0007, -2.0182],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0007, -2.0007, -2.0182],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0007, -2.0007, -2.0182],
          ...,
          [-0.9328, -1.0203, -1.1429,  ...,  0.6078,  0.6254,  0.6254],
          [-0.9503, -1.0378, -1.1604,  ...,  0.7479,  0.7479,  0.6078],
          [-0.9678, -1.1253, -1.1779,  ...,  0.6429,  0.5378,  0.5728]],

         [[-1.8044, -1.8044, -1.8044,  ..., -1.7347, -1.7347, -1.7870],
          [-1.8044, -1.8044, -

In [7]:
# ---------- Vision Model ----------
print("Analyzing chest X-ray using ResNet‑50 pretrained on ImageNet...")
model = models.resnet50(pretrained=True)
model.eval()

Analyzing chest X-ray using ResNet‑50 pretrained on ImageNet...
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\User/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


d:\MSc-github\msc-ir-code\ir_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\MSc-github\msc-ir-code\ir_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 97.8M/97.8M [01:27<00:00, 1.17MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
# Load ImageNet label names to decode class IDs
import urllib.request
LABELS_URL = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
imagenet_labels = urllib.request.urlopen(LABELS_URL).read().decode("utf-8").split("\n")

with torch.no_grad():
    outputs = model(image_tensor)
    probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
    top_probs, top_idxs = probabilities.topk(5)

top_predictions = [
    {
        "label": imagenet_labels[idx],
        "confidence": float(prob)
    }
    for prob, idx in zip(top_probs, top_idxs)
]

In [10]:
# Summaries for LLM
pred_summary_lines = [f"- {p['label']} ({p['confidence']*100:.2f}%)" for p in top_predictions]
vision_summary = "\n".join(pred_summary_lines)


In [11]:
pred_summary_lines

['- hourglass (63.93%)',
 '- beaker (10.42%)',
 '- plastic bag (1.96%)',
 '- Petri dish (1.78%)',
 '- fountain (1.21%)']

In [12]:
vision_summary

'- hourglass (63.93%)\n- beaker (10.42%)\n- plastic bag (1.96%)\n- Petri dish (1.78%)\n- fountain (1.21%)'

In [13]:
#---------- Prompt for Qwen ----------
prompt = f"""
You are a concise radiology assistant.

The vision model (ResNet‑50 pretrained on ImageNet) processed this chest X‑ray.
Top‑5 predicted visual categories with probabilities:

{vision_summary}

Based on this image, identify the **most likely lung disease** and list the **main possible causes**.

Respond briefly in **point form**, no more than **5 sentences total**, mentioning **only**:
- Disease name(s)
- Likely cause(s)

Avoid any extended explanations or extra commentary.
"""

In [14]:
import ollama


def call_ollama(model_name: str, prompt_text: str) -> str:
    """Call a locally installed Ollama model and return its response."""
    response = ollama.chat(
        model=model_name,
        messages=[{"role": "user", "content": prompt_text}]
    )
    return response['message']['content'].strip()



print("Generating interpretation with local Qwen‑1.8B...")
report_text = call_ollama(OLLAMA_MODEL, prompt)


Generating interpretation with local Qwen‑1.8B...


ResponseError: model 'qwen:1.8b' not found (status code: 404)

In [17]:
print(report_text)

Disease Name: Lung Cancer
Main Possible Causes:
1. smoking: The most common cause of lung cancer is smoking, which accounts for approximately 80% of all cases.
2. exposure to radon: Radon gas is emitted by radioactive sources such as nuclear power plants and uranium mines. Exposure to radon over a long period can increase the risk of developing lung cancer.
3. certain chemicals: Some chemicals, such as asbestos fibers, can cause lung cancer by depositing them in the lung tissue and triggering inflammation and scarring.
4. other underlying conditions: Other underlying conditions that may increase the risk of developing lung cancer include autoimmune disorders, such as bronchitis or asthma, and viral infections, such as the common cold or flu, which can trigger inflammation and scarring, leading to an increased risk of developing lung cancer.
In conclusion, lung cancer is a serious disease that can be caused by various underlying conditions. The most likely cause of lung cancer is smokin